In [ ]:
import asyncio
import websockets
import ssl
import json
from datetime import datetime
import argparse
import signal

# Handle asyncio event loop re-entry in Jupyter or similar environments
import nest_asyncio
nest_asyncio.apply()

# Global variable to handle cancellation
loop = asyncio.get_event_loop()
stop_event = asyncio.Event()

async def connect_to_websocket(uri, output_file):
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    websocket = None
    file = None
    epoch = 0

    try:
        websocket = await websockets.connect(uri, ssl=ssl_context, ping_interval=None)

        # Send the request to start the stream
        start_stream_message = json.dumps({"streamReadings": 0})
        await websocket.send(start_stream_message)

        file = open(output_file, 'w')

        while not stop_event.is_set():
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=1)
                processed_message = process_message(message)
                epoch += 1
                if processed_message:
                    if epoch % 50 == 0:
                        print(f"{processed_message}")
                    file.write(processed_message + "\n")
            except asyncio.TimeoutError:
                continue
            except websockets.exceptions.ConnectionClosedError as e:
                print(f"Connexion fermée : {e}")
                break  # Exit the loop on disconnection
            except Exception as e:
                print(f"Erreur lors de la réception du message : {e}")
                break  # Exit the loop on other errors
    except asyncio.CancelledError:
        print("Task was cancelled, cleaning up...")
    except Exception as e:
        print(f"Erreur de connexion : {e}")
    finally:
        # Ensure the WebSocket connection is closed
        if websocket:
            await websocket.close()
            print("Connexion WebSocket fermée.")

        # Ensure the file is closed
        if file:
            file.close()
            print("Fichier de sortie fermé.")

def process_message(message):
    try:
        data = json.loads(message)
        if 'readings' in data:
            readings = data['readings']
            if readings['runMode'] != "ORGASM_MODE":
                return None
            processed_message = json.dumps(readings)
        else:
            processed_message = json.dumps(data)
    except json.JSONDecodeError:
        processed_message = message

    return processed_message

def shutdown(signal_received, frame):
    print("Signal reçu pour l'arrêt.")
    stop_event.set()
    for task in asyncio.all_tasks(loop):
        task.cancel()

if __name__ == "__main__":
    uri = "wss://" + "192.168.1.19"
    
    print(f"uri = {uri}")
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"stream_output_{current_time}.txt"

    # Register the signal handler
    signal.signal(signal.SIGINT, shutdown)
    signal.signal(signal.SIGTERM, shutdown)

    try:
        loop.run_until_complete(connect_to_websocket(uri, output_file))
    except asyncio.CancelledError:
        pass
    finally:
        if not loop.is_running():
            loop.close()
        print("Programme arrêté.")


In [ ]:
import emd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import OneHotEncoder

# Load data from JSON, ignoring lines with "wifiStatus"
with open('G:\My Drive\Colab Notebooks\stream_output_20240902_085854.txt', 'r') as f:
#with open('/content/drive/MyDrive/Colab Notebooks/stream_output_20240902_085854.txt', 'r') as f:
  data = []
  for line in f:
    try:
      item = json.loads(line)
      if 'pressure' in item:  # Assuming your relevant data has "pressure" key
        data.append(item)
    except json.JSONDecodeError:
      pass  # Ignore lines that are not valid JSON

# Now you have a list of dictionaries with relevant data
# Extract pressure values
pressure_values = [item['pressure'] for item in data]
# Extract pressure values
pressure_values = [item['pressure'] for item in data]

window_size = 100  # Adjust as needed
moving_avg = np.convolve(pressure_values, np.ones(window_size), 'same') / window_size
pressure_values_normalized = pressure_values - moving_avg

# Create an EMD object and extract IMFs
imf = emd.sift.sift(np.array(pressure_values_normalized))
imfs = imf[:, 1:]

# Extract other features from JSON data
# Normalize motor values to the range [0, 1]
motor_values = np.array([item['motor'] for item in data]) / 255.0

runMode_values = np.array([item['runMode'] for item in data])
# One-hot encode runMode
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for numpy array
runMode_encoded = encoder.fit_transform(runMode_values.reshape(-1, 1))
arousal = np.array([item['arousal'] for item in data])

# Convert string booleans to integers
permitOrgasm_values = np.array([1 if item['permitOrgasm'] == 'true' else 0 for item in data])

# Create a NumPy array with IMFs and other features
# Ensure all feature arrays have the same length
min_length = min(len(imfs), len(motor_values), len(runMode_values), len(permitOrgasm_values))
#dataset = np.hstack((imfs[:min_length],
#                      motor_values[:min_length].reshape(-1, 1),
#                      runMode_encoded[:min_length],
#                      permitOrgasm_values[:min_length].reshape(-1, 1)))
dataset = np.hstack((imfs,
                      motor_values.reshape(-1, 1),
                      ))


# Print the shape of the dataset
print(dataset.shape, imfs.shape, motor_values.reshape(-1, 1).shape)

# Plot the IMFs and motor speed
num_imfs = imfs.shape[1]
fig, axes = plt.subplots(num_imfs + 2, 1, figsize=(12, 20))

# Plot IMFs
for i in range(num_imfs):
  axes[i].plot(imfs[:, i])
  axes[i].set_title(f"IMF {i+1}")

# Plot motor speed in the last subplot
axes[num_imfs].plot(motor_values)
axes[num_imfs].set_title('Motor Speed')

# Plot arousal in the last subplot
axes[num_imfs+1].plot(arousal)
axes[num_imfs+1].set_title('arousal')
plt.tight_layout()
plt.show()